In [16]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'lib')))


In [19]:

# New code to run Newton-Raphson fixed-point search using functions from the uploaded module.
# This cell will:
# - import the existing renormalization module (renormalization_cell135_mp.py)
# - define a Newton solver that builds finite-difference Jacobian (D), forms (D - I),
#   and solves (D - I) delta = J' - J to update J.
# - choose an initial J vector from find_J_c (the code's binary-search critical J) as default start.
# - estimate and print a conservative required chain length for computing up to max_r.
#
# IMPORTANT: This implementation uses numpy.linalg.solve for the linear solve (float64).
# For production/high-precision runs you can replace the solve with mpmath matrix solve.
# The code is designed to be drop-in with the repository file uploaded by the user.

from importlib import import_module
import numpy as np
from math import isfinite

# Import the uploaded module
import renormalization_cell135_mp as rmod
from mpmath import mp

mp.dps = 40

def build_J_func_from_vector(J_vec):
    """Return a J_func(d) that maps distance -> mp.mpf value from vector J_vec (1-indexed distances)."""
    J_dict = {i+1: mp.mpf(J_vec[i]) for i in range(len(J_vec))}
    return lambda d: J_dict.get(d, mp.mpf(0)) if d > 0 else mp.mpf(0)

def compute_J_vector(J_vec, max_r):
    """Compute J' vector of length max_r for given J_vec using compute_J_prime_func from uploaded module."""
    J_func = build_J_func_from_vector(J_vec)
    Jp = []
    for r in range(1, max_r+1):
        start = 3*r + 1
        val = rmod.compute_J_prime_func(start, J_func)
        Jp.append(mp.mpf(val))
    return Jp

def finite_difference_D(J_vec, max_r, eps_rel=1e-7):
    """Build Jacobian D_{ij} = d J'_i / d J_j by central finite differences.
       Returns a numpy array of shape (n,n) as float64 for solving.
    """
    n = len(J_vec)
    D = np.zeros((n,n), dtype=float)
    base_Jp = compute_J_vector(J_vec, max_r)
    for j in range(n):
        eps = max(abs(float(J_vec[j]))*eps_rel, eps_rel)
        Jp_plus = list(J_vec); Jp_minus = list(J_vec)
        Jp_plus[j] = mp.mpf(Jp_plus[j]) + mp.mpf(eps)
        Jp_minus[j] = mp.mpf(Jp_minus[j]) - mp.mpf(eps)
        Jprime_plus = compute_J_vector(Jp_plus, max_r)
        Jprime_minus = compute_J_vector(Jp_minus, max_r)
        for i in range(n):
            D[i,j] = float((Jprime_plus[i] - Jprime_minus[i])/(2*mp.mpf(eps)))
    return D, [float(x) for x in base_Jp]

def estimate_chain_length(max_r):
    """Conservative estimate: compute_J_prime_func uses right_pos = start, start+2, start+4 where start = 3*r + 1.
       The largest index accessed is start+4 = 3*r + 5 for r = max_r.
       So require chain length >= 3*max_r + 5. Return that.
    """
    return 3*max_r + 5

def newton_fixed_point(a, max_r=10, tol=1e-10, max_iter=20, eps_rel=1e-7, damp=1.0, verbose=True):
    """
    Perform Newton-Raphson to find fixed point J* in n-dimensional truncated coupling space up to distance max_r.
    - a: power-law exponent (called n in the user's code/paper)
    - max_r: number of distances kept (dimension n)
    - tol: tolerance for ||J' - J|| (infty-norm) to stop
    - max_iter: maximum Newton iterations
    - eps_rel: relative epsilon for finite-difference Jacobian
    - damp: damping factor for Newton updates (0<damp<=1)
    """
    # 1) Estimate required chain length
    chain_len = estimate_chain_length(max_r)
    if verbose:
        print(f"Estimated minimum chain length to compute up to r={max_r}: {chain_len} spins (conservative).")

    # 2) Get initial guess J0 from find_J_c in the uploaded module (uses n parameter named 'n' there)
    try:
        Jc = rmod.find_J_c(mp.mpf(a), max_k=max_r, tol=1e-6, J_low=1e-2, J_high=1e4)
        J0_val = Jc
        if verbose:
            print(f"Using critical J_c from find_J_c as initial J0 = {J0_val}")
    except Exception as e:
        # Fall back to J0=1.0 if find_J_c fails
        J0_val = mp.mpf(1.0)
        if verbose:
            print(f"find_J_c failed; falling back to J0 = {J0_val}. Error: {e}")

    # Initialize truncated vector J (length max_r)
    J_vec = [mp.mpf(rmod.get_J(r, J0_val, mp.mpf(a))) for r in range(1, max_r+1)]
    if verbose:
        print("Initial J vector (first 6 entries):", [str(x) for x in J_vec[:min(6, len(J_vec))]])

    for it in range(1, max_iter+1):
        # compute J' at current J
        Jprime_mp = compute_J_vector(J_vec, max_r)
        # residual F = J' - J
        F = [Jprime_mp[i] - mp.mpf(J_vec[i]) for i in range(max_r)]
        # check convergence
        res_norm = max([abs(float(x)) for x in F])
        if verbose:
            print(f"Newton iter {it}: residual infinity-norm = {res_norm:.3e}")
        if res_norm < tol:
            if verbose:
                print("Converged to tolerance.")
            return {'J_star': J_vec, 'residual': res_norm, 'iterations': it, 'converged': True}

        # Build Jacobian D by finite differences (expensive)
        D, base_Jp = finite_difference_D(J_vec, max_r, eps_rel=eps_rel)
        # Build M = D - I
        M = D - np.eye(max_r)
        # Right-hand side is F (as floats)
        rhs = np.array([float(x) for x in F], dtype=float)
        # Solve M * delta = rhs  -> delta = M^{-1} rhs
        try:
            delta = np.linalg.solve(M, rhs)
        except np.linalg.LinAlgError as e:
            if verbose:
                print("Linear solve failed (singular or ill-conditioned M). Returning current iterate.")
            return {'J_star': J_vec, 'residual': res_norm, 'iterations': it, 'converged': False, 'error': str(e)}

        # Update J (apply damping)
        for i in range(max_r):
            J_vec[i] = mp.mpf(J_vec[i]) - mp.mpf(damp*delta[i])

    # End iterations
    if verbose:
        print("Reached maximum iterations without converging.")
    return {'J_star': J_vec, 'residual': res_norm, 'iterations': max_iter, 'converged': False}

In [20]:

# Run an example for a=1, max_r=10
result = newton_fixed_point(a=1, max_r=10, tol=1e-5, max_iter=8, eps_rel=1e-7, damp=0.8, verbose=True)

# Display a few entries of the resulting fixed-point candidate
print("\nResult summary:")
print("Converged:", result['converged'])
print("Iterations:", result['iterations'])
print("Residual:", result['residual'])
print("First 8 entries of J* (mpf):")
for i, val in enumerate(result['J_star'][:8]):
    print(f" r={i+1}: {val}")


Estimated minimum chain length to compute up to r=10: 35 spins (conservative).
Using critical J_c from find_J_c as initial J0 = 0.7987182553479215132291426776566509109172
Initial J vector (first 6 entries): ['0.7987182553479215132291426776566509109172', '0.3993591276739607566145713388283254554586', '0.2662394184493071710763808925522169703057', '0.1996795638369803783072856694141627277293', '0.1597436510695843026458285355313301821834', '0.1331197092246535855381904462761084851529']
Newton iter 1: residual infinity-norm = 9.656e-01
Newton iter 2: residual infinity-norm = 3.124e-01
Newton iter 3: residual infinity-norm = 1.150e+02
Linear solve failed (singular or ill-conditioned M). Returning current iterate.

Result summary:
Converged: False
Iterations: 3
Residual: 115.00953056643473
First 8 entries of J* (mpf):
 r=1: -179290154.0437722078116467015057753236841
 r=2: -59.66877477773975502185061769360290076825
 r=3: -2.394009100974152019237582170538124195356
 r=4: -0.962897250150665587279528